In [16]:
# Initial imports
import os
import requests # not in use for pulling stock price
import requests_html # not in use for pulling stock price
import pandas as pd
import numpy as np
from numpy import inf

from pathlib import Path

from dotenv import load_dotenv

# Yahoo_fin api to pull adjusted close price
import yahoo_fin.stock_info as si
from datetime import date, timedelta
import datetime
# from yahoo_earnings_calendar import YahooEarningsCalendar

# Alternative to pulling the data for earnings calls without yahoo earnings, which does not work
import json
from bs4 import BeautifulSoup

In [17]:
# We use yahoo_fin API to pull adjusted closing prices. We tried Alpaca API, but priecs are not adjusted for stock splits
# We define the tickers and other inputs for the API
ticker_list = ['TSLA', 'QQQ']
start_date='2010-06-29'
end_date=date.today()#'2021-04-09'
index_as_date=True
interval='1d'


# we create an empty data frame where we will store the final data
closing_prices_df = pd.DataFrame()

# Yahoo fin stores data in dictionary. We will create a for loop to pull data for all the tickers we need. We will use the same loop to create the data frame
# We create an empty dictionary where we will store data from the API
historical_data = {}

for ticker in ticker_list:
    historical_data[ticker] = si.get_data(ticker,start_date,end_date,index_as_date,interval)
    closing_prices_df[ticker]=historical_data[ticker]['adjclose']

# Add a title to index
closing_prices_df.index.name = 'date'

In [18]:
date_from = datetime.datetime.strptime('Jun 29 2020  10:00AM', '%b %d %Y %I:%M%p')
date_to = datetime.datetime.strptime('Apr 09 2021  10:00AM', '%b %d %Y %I:%M%p')
type(date_from)
date_to

datetime.datetime(2021, 4, 9, 10, 0)

In [19]:
# Funciton to pull earnings dates for any stock
# uses Beautiful Soup to scrape data from yahoo finance

def my_earnings_dates(symbol, start_date, end_date):

    #     Define the URL
    url = f"https://finance.yahoo.com/calendar/earnings?from={start_date}&to={end_date}&symbol={symbol}"

    #     prepare start and end date to pass if conditions inside funciton only
    start_date=datetime.datetime.strptime(str(start_date), '%Y-%m-%d')
    end_date=datetime.datetime.strptime(str(end_date), '%Y-%m-%d')
    
    #     Get the data from the URL and use beautiful soup to parse it 
    response_data = requests.get(url)
    soup = BeautifulSoup(response_data.text)
    
    #     Find all the earnings dates associated to the symbol and date
    date_zone=soup.find_all('td',attrs={'aria-label':'Earnings Date'})

    #     start populating the dates and time zones with a for loop
    date_list=[]
    time_zone_list=[]
    
    
    for i in range(0,len(date_zone)):
        
        #     print(all_data[i].text)
        #     Pull the date and time for each earnings call
        date_time=date_zone[i].find_all('span')[0]
        date_time=date_time.text
        date_time=datetime.datetime.strptime(date_time, '%b %d, %Y, %I %p')
         
        if date_time>=start_date and date_time<=end_date:
            #     we convert the format into a time string
            date_time=date_time.strftime('%Y-%m-%d %I:%M%p')

            #     Pull the time zone
            time_zone=date_zone[i].find_all('span')[1]
            time_zone=time_zone.text

            #     append date_time and time_zone 
            date_list.append(date_time)
            time_zone_list.append(time_zone)
    
    
    #    Create data frame with output data
    earnings_dates=pd.DataFrame()
    earnings_dates['date']=date_list
    earnings_dates['time zone']=time_zone
    earnings_dates.sort_values(by=['date'], inplace=True)
    
    return earnings_dates.sort_values(by=['date'])

# Pull earnings dates
# Define ticker
# Start and end date are defined above
symbol= 'TSLA'
# call funciton
earnings_dates_df=my_earnings_dates(symbol, start_date, end_date)

# Make sure to adjust data if needed - adjustment by market hour 

# we change the name to date column - we will drop this field later. We need a date field that shows off market hour tweets as t+1 
earnings_dates_df.rename(columns={'date':'date original'},inplace=True)
earnings_dates_df['date original']=pd.to_datetime(earnings_dates_df['date original'])

# Make earnings calls after 4pm fall into the following day
# Define market hour limit as everything after 16hs 00 min 00 sec
min_hour=16
min_minute=0
min_second=0

# we create the new field equalt to date original 
earnings_dates_df['date']=earnings_dates_df['date original'].copy()

# we add 1 day to date original if the tweet occured off market hours
earnings_dates_df.loc[(earnings_dates_df['date original'].dt.hour>=min_hour) & (earnings_dates_df['date original'].dt.minute>min_minute) & (earnings_dates_df['date original'].dt.second>min_second), 'date'] = earnings_dates_df['date original']+timedelta(days=1)


#Create Flag 1,0 if there was an earnings call on that day
# it will be one in this data set
earnings_dates_df['earnings flag']=1
# earnings_dates_df['earnings flag'].astype('int')

# drop unused fields
earnings_dates_df.drop(columns={'time zone', 'date original'}, inplace=True)
earnings_dates_df.set_index('date', inplace=True)


earnings_dates_df.head()

,earnings flag
date,
2010-11-09,1
2011-02-15,1
2011-05-04,1
2011-08-03,1
2011-11-02,1


In [20]:
# Clean the format of the dates
# closing_prices_df.index = closing_prices_df.index.date

# We use earnings data to create a flag for the days with earnings calls 
closing_prices_df=closing_prices_df.join(earnings_dates_df, how='outer')

# Fill in N/As with 0: we create a boolean variable
closing_prices_df['earnings flag']=closing_prices_df['earnings flag'].fillna(0)

# Change format to integer
closing_prices_df['earnings flag']=closing_prices_df['earnings flag'].astype('int')

closing_prices_df.tail()

,TSLA,QQQ,earnings flag
date,,,
2021-04-06,691.619995,330.820007,0
2021-04-07,670.969971,331.619995,0
2021-04-08,683.799988,335.079987,0
2021-04-09,677.020020,337.109985,0
2021-04-12,701.979980,336.670013,0


In [21]:
closing_prices_df.drop(columns = 'QQQ')

,TSLA,earnings flag
date,,
2010-06-29,4.778000,0
2010-06-30,4.766000,0
2010-07-01,4.392000,0
2010-07-02,3.840000,0
2010-07-06,3.222000,0
2010-07-07,3.160000,0
2010-07-08,3.492000,0
2010-07-09,3.480000,0
2010-07-12,3.410000,0


In [22]:
closing_prices_df = closing_prices_df.diff()

In [23]:
TSLA_prices_df = closing_prices_df

In [24]:
TSLA_prices_df.fillna(0)

,TSLA,QQQ,earnings flag
date,,,
2010-06-29,0.000000,0.000000,0.0
2010-06-30,-0.012000,-0.593243,0.0
2010-07-01,-0.374000,-0.107872,0.0
2010-07-02,-0.552000,-0.107841,0.0
2010-07-06,-0.618000,0.116844,0.0
2010-07-07,-0.062000,1.222408,0.0
2010-07-08,0.332000,0.215714,0.0
2010-07-09,-0.012000,0.377499,0.0
2010-07-12,-0.070000,0.116852,0.0


In [25]:
print(type(TSLA_prices_df))

<class 'pandas.core.frame.DataFrame'>


In [26]:
TSLA_prices_df[TSLA_prices_df == -inf] = 0
TSLA_prices_df.dropna()

,TSLA,QQQ,earnings flag
date,,,
2010-06-30,-0.012000,-0.593243,0.0
2010-07-01,-0.374000,-0.107872,0.0
2010-07-02,-0.552000,-0.107841,0.0
2010-07-06,-0.618000,0.116844,0.0
2010-07-07,-0.062000,1.222408,0.0
2010-07-08,0.332000,0.215714,0.0
2010-07-09,-0.012000,0.377499,0.0
2010-07-12,-0.070000,0.116852,0.0
2010-07-13,0.218000,0.521317,0.0


In [27]:
TSLA_prices_df['earnings flag'] = np.where(TSLA_prices_df['TSLA']>0,1,0)

In [28]:
TSLA_prices_df.tail()

In [34]:
TSLA_prices_df=TSLA_prices_df.drop(columns='QQQ')

In [35]:
TSLA_prices_df.to_csv('TSLA_prices_df.csv')

In [36]:
TSLA_prices_df.head()

,TSLA,earnings flag
date,,
2010-06-29,NaN,0
2010-06-30,-0.012,0
2010-07-01,-0.374,0
2010-07-02,-0.552,0
2010-07-06,-0.618,0
